In [1]:
# Import necessary libraries
import pandas as pd
from sklearn.ensemble import GradientBoostingRegressor
# Read the original data from 'data.csv' into a Pandas DataFrame
data = pd.read_csv('data.csv')

# Define columns to process by excluding 'player_name' and 'last_match'
columns_to_process = data.columns.difference(['player_name', 'last_match'])

# Define a function to preprocess non-numeric values in the dataset
def preprocess_non_numeric(value):
    if pd.notna(value):
        value = str(value)
        if value.isnumeric():
            return int(value)
        elif '*' in value:
            return int(value.split('*')[0])
        elif value.lower() in ['dnb', 'tdnb', '-', 'absent']:
            return 0
    return value

# Apply the preprocessing function to the specified columns
for col in columns_to_process:
    data[col] = data[col].apply(preprocess_non_numeric)


# Save the preprocessed data to 'preprocessed_data.csv' without the index
data.to_csv('preprocessed_data.csv', index=False)

# Read the preprocessed data into a Pandas DataFrame
data = pd.read_csv('preprocessed_data.csv')

# Define X and y for the machine learning model
X = data[['player_id']]
y_runs = data['runs_scored']  # Separate target for runs
y_wickets = data['wickets']   # Separate target for wickets

# Create separate Gradient Boosting Regressor models for runs and wickets
model_runs = GradientBoostingRegressor(n_estimators=100, random_state=42)
model_wickets = GradientBoostingRegressor(n_estimators=100, random_state=42)

# Fit the models to the data
model_runs.fit(X, y_runs)
model_wickets.fit(X, y_wickets)
# Make predictions using the trained models
predicted_runs = model_runs.predict(X)
predicted_wickets = model_wickets.predict(X)
# Create DataFrames with player_id, predicted_runs, and predicted_wickets
predicted_data = pd.DataFrame({'player_id': data['player_id'], 'predicted_runs': predicted_runs, 'predicted_wickets': predicted_wickets})

# Save the predicted data to 'predicted_runs_and_wickets.csv' without the index
predicted_data.to_csv('predicted_runs_and_wickets.csv', index=False)
# Read the predicted data into a new Pandas DataFrame
predicted_data = pd.read_csv('predicted_runs_and_wickets.csv')
# Group the data by 'player_id' and calculate the mean of 'predicted_runs' and 'predicted_wickets'
aggregated_data = predicted_data.groupby(['player_id']).agg({
    'predicted_runs': 'mean',
    'predicted_wickets': 'mean'
}).reset_index()
# Rename the columns to 'runs' and 'wickets'
aggregated_data.rename(columns={'predicted_runs': 'runs', 'predicted_wickets': 'wickets'}, inplace=True)
# Round the 'runs' and 'wickets' columns and cast them to integers
aggregated_data['runs'] = aggregated_data['runs'].round().astype(int)
aggregated_data['wickets'] = aggregated_data['wickets'].round().astype(int)
# Save the aggregated data to 'aggregated_runs_and_wickets.csv' without the index
aggregated_data.to_csv('aggregated_runs_and_wickets.csv', index=False)
# Read the aggregated data into a new Pandas DataFrame
aggregated_data = pd.read_csv('aggregated_runs_and_wickets.csv')
# Define a list of player IDs that are not playing (12, 13, 14, 15, 19, 26, 27, 30)
non_playing_ids = [12, 13, 14, 15, 19, 26, 27, 30]
# Set runs and wickets to 0 for non-playing members
aggregated_data.loc[aggregated_data['player_id'].isin(non_playing_ids), 'runs'] = 0
aggregated_data.loc[aggregated_data['player_id'].isin(non_playing_ids), 'wickets'] = 0
# Save the final data with zero runs and wickets for non-playing members
aggregated_data.to_csv('final_aggregated_runs_and_wickets.csv', index=False)
aggregated_data.to_csv


<bound method NDFrame.to_csv of     player_id  runs  wickets
0           1    40        0
1           2    22        1
2           3    54        0
3           4    46        0
4           5    38        0
5           6    37        0
6           7    14        1
7           8     7        1
8           9     2        2
9          10     2        2
10         11     2        2
11         12     0        0
12         13     0        0
13         14     0        0
14         15     0        0
15         16    49        0
16         17    13        1
17         18    20        0
18         19     0        0
19         20     2        2
20         21     4        1
21         22    25        1
22         23    44        0
23         24    12        1
24         25    25        0
25         26     0        0
26         27     0        0
27         28    12        0
28         29     4        2
29         30     0        0>